## 1,レース情報をスクレイピング

jupyter notebook で [ImportError: lxml not found, please install it]エラーが出た場合は [pip3 install lxml] でインストール後プロジェクトを再起動


In [23]:
import pandas as pd
from tqdm import tqdm
import time

url >> race_id=2022 09 01 11 01 : 年  競馬場(01:札幌,02:函館,03:福島,04:新潟,05:東京,06:中山,07:中京,08:京都,09:阪神,10:小倉) 何回(今年その競馬場でのレース回数) 何日目(何レース目)

In [24]:
pd.set_option('display.max_rows',5)
pd.options.display.width = 200
pd.options.display.column_space = 100

スクレイピング

In [25]:
df = pd.read_pickle("df.pickle")
already = df.index.unique()
already = list(map(str, already))
already

['202103010101',
 '202103010102',
 '202103010103',
 '202103010104',
 '202103010105',
 '202103010106',
 '202103010107',
 '202103010108',
 '202103010109',
 '202103010110',
 '202103010111',
 '202103010112',
 '202101010101',
 '202101010102',
 '202101010103',
 '202101010104',
 '202101010105',
 '202101010106',
 '202101010107',
 '202101010108',
 '202101010109',
 '202101010110',
 '202101010111',
 '202101010112',
 '202102010101',
 '202102010102',
 '202102010103',
 '202102010104',
 '202102010105',
 '202102010106',
 '202102010107',
 '202102010108',
 '202102010109',
 '202102010110',
 '202102010111',
 '202102010112']

In [26]:
def scrape_race_results(race_id_lists,par_race_results={}):
  race_result_dict = par_race_results
  for _race_id in tqdm(race_id_lists):
    if _race_id in already:
      continue
    time.sleep(2)
    try:
      url = "https://race.netkeiba.com/race/result.html?race_id=" + _race_id
      race_result_dict[_race_id] = pd.read_html(url)[0]
      # print(race_result_dict[race_id])
    except UnicodeDecodeError as e:
      # print(e)
      # print(f"レースID: {race_id} のデータはありません")
      continue
    except KeyboardInterrupt:
      break
    except TimeoutError:
      break
  return race_result_dict

# race_result = {}

dictのkeyをrace_idに変換


In [35]:
def get_house_data():
  race_id_lists = []

  for 年 in range(2021,2022):
    for 競馬場 in range(1,5): # 1,11
      for 回 in range(1,2): # 1,11
        for 回目 in range(1,2): # 1,11
          for レース回 in range(1,13): # 1,13
            race_id = f"{年}{str(競馬場).zfill(2)}{str(回).zfill(2)}{str(回目).zfill(2)}{str(レース回).zfill(2)}"
            race_id_lists.append(race_id)
            
  race_result = scrape_race_results(race_id_lists)
  # print(race_result)
  # print(race_result.keys())
  for key in race_result.keys():
    # print(key)
    # print(race_result[key].index)
    race_result[key].index = [key] * len(race_result[key])
    
  results = pd.concat([race_result[key] for key in race_result.keys()],sort=False)
  return results

新規でdata取得するとき

In [36]:
house_data = get_house_data()

100%|██████████| 48/48 [00:27<00:00,  1.77it/s]

{'202104010101':     着順  枠  馬番        馬名  性齢    斤量   騎手     タイム     着差  人気  単勝オッズ   後3F コーナー通過順    厩舎   馬体重(増減)
0    1  5   9      テオドラ  牝3  53.0  ☆亀田  1:12.9    NaN   4    7.6  37.4     4-4  栗東北出  466(+10)
1    2  5   8  サイモンルピナス  牝3  54.0  吉田隼  1:13.1  1.1/4   2    3.4  38.0     3-3  栗東羽月   430(-4)
..  .. ..  ..       ...  ..   ...  ...     ...    ...  ..    ...   ...     ...   ...       ...
13  14  7  12      タカミズ  牝3  54.0   森裕  1:16.7      5   8   26.3  40.9     8-8  栗東吉田   422(-8)
14  15  8  15   リネンゼンシン  牝3  54.0   丸田  1:20.9      大  14  124.3  41.6   15-15  美浦南田    446(0)

[15 rows x 15 columns], '202104010102':     着順  枠  馬番         馬名  性齢    斤量    騎手     タイム   着差  人気  単勝オッズ   後3F      コーナー通過順     厩舎   馬体重(増減)
0    1  4   7    プラウドルック  牝3  54.0   鮫島駿  1:55.5  NaN   1    2.9  39.5      3-3-3-3  栗東斉藤崇   444(+4)
1    2  7  12  メイショウコトダマ  牡3  56.0    城戸  1:55.5  アタマ   8   39.9  39.8      1-1-1-1   栗東荒川  452(-10)
..  .. ..  ..        ...  ..   ...   ...     ...  ...  ..    ...   ..

追加でdata取得するとき

In [232]:
len(house_data)

162

In [233]:
house_data

,着順,枠,馬番,馬名,性齢,斤量,騎手,タイム,着差,人気,単勝オッズ,後3F,コーナー通過順,厩舎,馬体重(増減)
202103010101,1,6,9,ビレッジイーグル,牡4,60.0,大江原,2:58.2,NaN,5,8.6,13.0,1-1-1-1,美浦竹内,500(-6)
202103010101,2,3,4,ベイビーステップ,牡7,60.0,伴,2:58.4,1,6,10.1,13.0,4-3-2-2,美浦菊川,496(0)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202103010112,15,7,14,トランザクト,牡3,52.0,◇藤田菜,1:10.6,1.3/4,5,14.8,39.3,4-7,美浦武藤,496(-4)
202103010112,16,4,8,リゼ,牝5,55.0,石川,1:12.4,大,14,107.5,39.5,15-15,美浦田中清,446(-4)


## csvで保存してデータの整理

In [237]:

house_data.to_csv('test1.csv')
# house_data.to_csv('test2.csv')

In [236]:
house_data.columns
print(house_data["着順"].value_counts())
print(house_data["枠"].value_counts())
print(house_data["馬番"].value_counts())
print(house_data["馬名"].value_counts())
print(house_data["性齢"].value_counts())
print(house_data["斤量"].value_counts())
print(house_data["騎手"].value_counts())
print(house_data["タイム"].value_counts())
print(house_data["着差"].value_counts())
print(house_data["人気"].value_counts())
print(house_data["単勝オッズ"].value_counts())
print(house_data["後3F"].value_counts())
print(house_data["コーナー通過順"].value_counts())
print(house_data["厩舎"].value_counts())
print(house_data["馬体重(増減)"].value_counts())

1     12
2     12
3     12
4     12
5     12
6     12
7     12
8     12
9     11
10    11
11    10
12    10
13     9
14     7
15     5
16     3
Name: 着順, dtype: int64
7    23
8    23
6    22
5    22
4    21
3    19
2    17
1    15
Name: 枠, dtype: int64
4     12
2     12
6     12
7     12
5     12
3     12
8     12
1     12
9     11
10    11
11    10
12    10
13     9
14     7
15     5
16     3
Name: 馬番, dtype: int64
ビレッジイーグル     1
ヴェルザスカ       1
ナーシサステソーロ    1
コーンフィールド     1
マベルロンジュ      1
キタサンマーベラス    1
ドリームインパクト    1
ナリノモンターニュ    1
シャイニングライト    1
スカイテラス       1
トーセンクライマー    1
ウインエアフォルク    1
ケイティミラクル     1
ラヴィアンレーヴ     1
ルーパステソーロ     1
ラズベリーラバー     1
クリノプレミアム     1
クールウォーター     1
ミアマンテ        1
ネオヴィットーリア    1
ズールー         1
ノットイェット      1
ギャラクサイト      1
ウインシュクラン     1
マイネルグロン      1
ハムレボ         1
ランドヘルツ       1
マイネルパリオート    1
ダノンマデイラ      1
スマイルフェアリー    1
ボーンジーニアス     1
スノームーン       1
グッドスペンサー     1
シゲルシャイン      1
ミラキュラスサイン    1
カフェテオドーラ     1
グラシアディヴィナ    1
オンリーグローリー    1
サンマルシップ   

In [ ]:
# pd.set_option('display.max_rows',None)
# df = pd.read_csv("test.csv")
# df1 = df.iloc[:,:15]
# print(df1.columns)
# df1

Index(['Unnamed: 0', '着順', '枠', '馬番', '馬名', '性齢', '斤量', '騎手', 'タイム', '着差', '人気', '単勝オッズ', '後3F', 'コーナー通過順', '厩舎'], dtype='object')


0       栗東大橋
1       栗東寺島
2       美浦深山
3      美浦伊藤大
4       栗東加用
5       美浦手塚
6       美浦田村
7      栗東飯田祐
8      美浦高橋裕
9       栗東梅田
10     栗東清水久
11      美浦手塚
12       美浦堀
13     栗東大根田
14     美浦田中博
15      美浦小島
16       栗東宮
17      栗東野中
18      栗東平田
19     栗東高柳大
20      栗東岡田
21      栗東牧浦
22     栗東高橋忠
23      栗東浜田
24     栗東池添兼
25      美浦新開
26     栗東中内田
27      栗東大橋
28      美浦竹内
29      美浦栗田
30     美浦大江原
31      美浦稲垣
32      美浦高木
33      栗東友道
34     美浦加藤征
35     栗東藤原英
36      栗東武英
37      栗東坂口
38      美浦岩戸
39      美浦深山
40     美浦清水英
41     栗東藤沢則
42      美浦田村
43     美浦伊藤伸
44      栗東吉田
45      美浦本間
46     栗東杉山佳
47     美浦伊藤圭
48      美浦杉浦
49      美浦小野
50      美浦新開
51      栗東浅見
52      美浦岩戸
53      栗東今野
54     美浦蛯名利
55      栗東加用
56     栗東池添兼
57      美浦堀井
58      栗東須貝
59      栗東松下
60     栗東安田隆
61      栗東音無
62      美浦国枝
63      栗東本田
64      栗東笹田
65      美浦堀井
66     美浦田中清
67     美浦古賀慎
68      美浦杉浦
69       栗東宮
70      栗東渡辺
71      栗東吉村
72     美浦田中博
73      栗東森田
74     栗東高柳大
75     美浦高橋祥
76      栗東新谷

columnに余計なものがないか

値が正確に入っていない